In [4]:
import requests
from bs4 import BeautifulSoup
from threading import Timer

def create_dicts():
    #user input
    usr_DeptNo = input('輸入科系代碼：')
    usr_CrsNo = input('輸入科目代碼：')

    #get the main page of class-signing
    url = 'http://course-query.acad.ncku.edu.tw/qry'
    response = requests.get(url)
    response.encoding = 'UTF-8'
    soup = BeautifulSoup(response.text , 'html.parser')
    # print(soup)

    #create a dictionary in the form of {department_No. : site_link} for each department
    dept_all = soup.find('ul',id='dept_list')
    dept = dept_all.find_all('div', 'dept')
    dept_link = []
    for i in dept:
        dept_link.append("http://course-query.acad.ncku.edu.tw/qry/" + i.find('a').get('href'))
    dept_text = []
    for i in dept:
        dept_text.append(i.string)
    dept_no = []
    for string in dept_text:
        dept_no.append(string[string.find('(')+2 : string.find(u' ）')])
    dept_dict = dict(zip(dept_no, dept_link))
    # print(dept_dict)

    #create a dictionary in the form of {course_No. : [course_name, Space_Available]}
    url = dept_dict[usr_DeptNo]
    response = requests.get(url)
    response.encoding = 'UTF-8'
    soup = BeautifulSoup(response.text , 'html.parser')
    course_tmp = []
    course_all = []
    course_tmp = soup.find_all('td')
    for row in range(1,int(len(course_tmp)/24)+1):
        course_all.append(course_tmp[24*(row-1):24*row-1])
    course_dict = {}
    for i in course_all:
        if i[2].string == None:
            continue
        course_dict.update({i[2].string: [i[10].string, i[15].string]})
        
    #set up a loop timer
    

In [5]:
def Looper():
    timer = Timer(3, create_dicts)
    timer.start()
    print("{0:s}，餘額: {1:s}".format(course_dict[usr_CrsNo][0], course_dict[usr_CrsNo][1]))

輸入科目代碼：106
音樂文化導引，餘額: 額滿
